In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np
import os
from google.colab import drive
from collections import Counter

In [2]:
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/TR_DIMA/Hybrid-SC')

from model import ResNet50DualBranch, CombinedPSCLoss

Mounted at /content/drive


In [3]:
# Path to the dataset and model
path_train_dataset = "/content/drive/MyDrive/TR_DIMA/training_set_reduit"

base_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

augmented_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.GaussianBlur(kernel_size=9, sigma=(0.01, 5)),
        transforms.RandomResizedCrop(size=224, scale=(0.7, 1.0), ratio = (0.2,5)),
        transforms.ToTensor(),
    ])



dataset_init = datasets.ImageFolder(root=path_train_dataset, transform=augmented_transform)

train_size = int(0.8 * len(dataset_init))
valid_size = len(dataset_init) - train_size

train_dataset, valid_dataset = random_split(dataset_init, [train_size, valid_size])



In [4]:
# Sampler for class balancing

train_indices = train_dataset.indices
train_labels = [dataset_init.samples[i][1] for i in train_indices]

class_counts = Counter(train_labels)
class_weights = {label: 1.0 / count for label, count in class_counts.items()}
sample_weights = [class_weights[label] for label in train_labels]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples= len(sample_weights), replacement=True)

# Create DataLoaders for training and validation datasets

train_loader = DataLoader(train_dataset, batch_size=56, sampler=sampler, num_workers=16, pin_memory=True)
val_loader = DataLoader(valid_dataset, batch_size=56, shuffle=False, num_workers=16, pin_memory=True)

valid_classes = dataset_init.classes

#Initialisation model

model = ResNet50DualBranch(num_classes=len(valid_classes), proj_dim=128, hidden_dim=2048).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

#Loss function

criterion = CombinedPSCLoss(alpha=1.0, temperature=0.1).cuda()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=Res

In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100):

    for epoch in range(num_epochs):
        model.train()
        alpha = 1.0 - (epoch / num_epochs)**2  # Linear decay of alpha
        running_loss = 0.0
        correct = 0
        total = 0
        criterion = CombinedPSCLoss(alpha=alpha, temperature=0.1).cuda()

        for images, labels in tqdm(train_loader):
            images, labels = images.cuda(), labels.cuda()

            optimizer.zero_grad()
            out_cls, out_proj, proto = model(x_cls=images, x_proj=images)
            loss = criterion(out_cls, out_proj, labels, proto)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(out_cls.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.cuda(), labels.cuda()
                out_cls, out_proj, proto = model(x_cls=images, x_proj=images)
                loss = criterion(out_cls, out_proj, labels, proto)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(out_cls.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = val_correct / val_total
        print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.4f}")



    return model

In [9]:
model_trained=train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)

  0%|          | 0/692 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 692/692 [01:40<00:00,  6.87it/s]

Epoch [1/100], Loss: -7.8846, Accuracy: 0.9749


Validation Loss: -7.0856, Validation Accuracy: 0.9345


100%|██████████| 692/692 [01:40<00:00,  6.85it/s]

Epoch [2/100], Loss: -7.9009, Accuracy: 0.9753


...

Epoch [99/100], Loss: -0.3056, Accuracy: 0.9943


Validation Loss: -0.1615, Validation Accuracy: 0.9623


100%|██████████| 692/692 [01:40<00:00,  6.87it/s]

Epoch [100/100], Loss: -0.1428, Accuracy: 0.9941


Validation Loss: 0.0023, Validation Accuracy: 0.9614


In [10]:
torch.save(model.state_dict(), "/content/drive/MyDrive/TR_DIMA/Entrainement/hybridsc.pth")


In [12]:
backbone_classifier_state = {
    'backbone': model.backbone.state_dict(),
    'classifier': model.classifier.state_dict()
}

torch.save(backbone_classifier_state, "/content/drive/MyDrive/TR_DIMA/Entrainement/hybridsc_V2.pth")